<a href="https://colab.research.google.com/github/waynekearns/90DaysOfDevOps/blob/main/Copy_of_Wayne_project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Start Here
To start we need to clone the git repo of the google API python cliet as well as the CBN Tool and install it on this Google Colab instance.

In [ ]:
%%shell

rm -rf sample_data
git clone https://github.com/googleapis/google-api-python-client
git clone https://github.com/chronicle/cbn-tool
pip install -r /content/cbn-tool/requirements.txt
pip install google-api-python-client

The next step is to upload the appropriate service key to the environment and initialize the web client.   

> ⇒ Click the play button below, then select your keyfile.



In [ ]:
#@title Upload keyfile & Initialize web client
# Imports required for the sample - Google Auth and API Client Library Imports.
# Get these packages from https://pypi.org/project/google-api-python-client/ or run $ pip
# install google-api-python-client from your terminal
from google.oauth2 import service_account
from googleapiclient import _auth
import urllib
import json
import base64
import http
import re
import os
import sys
# import subprocess
from pprint import pprint
from time import sleep
from urllib.parse import urlencode

SCOPES = ['https://www.googleapis.com/auth/chronicle-backstory', 'https://www.googleapis.com/auth/malachite-ingestion']
#@markdown Please make sure you select the appopriate region below.
region = "North America" #@param ["North America", "Europe", "Asia"]

if region == "North America":
  region_prefix = ""
  cbn_region = "US"
elif region == "Europe":
  region_prefix = "europe-"
  cbn_region = "EUROPE"
elif region == "Asia":
  region_prefix = "asia-southeast1-"
  cbn_region = "ASIA"

# The apikeys-demo.json file contains the customer's OAuth 2 credentials.
# SERVICE_ACCOUNT_FILE is the full path to the apikeys-demo.json file
# ToDo: Replace this with the full path to your OAuth2 credentials

from google.colab import files
print("Please select and upload your keyfile")
uploaded = files.upload()

os.system(f"mv {list(uploaded.keys())[0]} mssp_key.json")

key_filename = "mssp_key.json"

SERVICE_ACCOUNT_FILE = key_filename

# Create a credential using Google Developer Service Account Credential and Chronicle API
# Scope.
credentials = service_account.Credentials.from_service_account_file(SERVICE_ACCOUNT_FILE, scopes=SCOPES)

# Build an HTTP client to make authorized OAuth requests.
http_client = _auth.authorized_http(credentials)

print("Webclient initialized")

In [ ]:
#@title Grab MSSP customer keys.
#@markdown This needs to be run for subsequent cells to function properly!
download_output = False 

mssp_customers = []

uri_to_get = f"https://{region_prefix}backstory.googleapis.com/v1/partner/customer/listcustomers"
# send the request and process the response
resp = http_client.request(uri_to_get, "GET")
json_resp = json.loads(resp[1])
if resp[0].status != http.HTTPStatus.OK:
  pprint(json_resp.get('error').get('message'))
else:
  customers = json_resp
  for customer in customers['customers']:
    # we only seem to take the lower case version of the customer name to force it to lower
    mssp_customers.append(customer["customerCode"].lower())
# pprint(mssp_customers)
  
for customer_code in mssp_customers:
  decode_base64 = True

  uri_to_get = f"https://{region_prefix}backstory.googleapis.com/v1/partner/customer/getcustomer?customer_code={customer_code}"
  # send the request and process the response
  resp = http_client.request(uri_to_get, "GET")
  json_resp = json.loads(resp[1])
  if resp[0].status != http.HTTPStatus.OK:
    pprint(json_resp.get('error').get('message'))
  else:
    customer_data = json_resp
    if decode_base64:
      for credential_index in range(len(customer_data['credentials'])):
        customer_data['credentials'][credential_index]['credential'] = base64.b64decode(customer_data['credentials'][credential_index]['credential']).decode()
      customer_credentials = customer_data['credentials']
      printable_customer_data = customer_data.copy()
      if 'ssoConfig' in printable_customer_data:
        del printable_customer_data['ssoConfig']
      del printable_customer_data['credentials']
      # pprint(printable_customer_data)
      for credential in customer_credentials:
        if "BACKSTORY" not in credential['credentialType']:
          continue
        with open(f"{customer_code}.json", "w") as credential_file:
          print(f"Writing Backstory API key for customer code {customer_code}")
          credential_file.write(credential["credential"])
        
      print(f"Available customers to work on: {', '.join(mssp_customers)}")
    else:
      pprint(customer_data)


Writing Backstory API key for customer code dmrsktst
Available customers to work on: dmrsktst


In [ ]:
#@title Get a single tenant's rules
tenant_code = "dmrsknfr" #@param {type:"string"}
#@markdown If selected a zip file with all the rules for this tenant will be downloaded.
download_zipped_data = True #@param {type:"boolean"}

SERVICE_ACCOUNT_FILE = f"{tenant_code}.json"
credentials = service_account.Credentials.from_service_account_file(SERVICE_ACCOUNT_FILE, scopes=SCOPES)
http_client = _auth.authorized_http(credentials)

if f"{tenant_code}_rules" not in os.listdir():
  os.mkdir(f"{tenant_code}_rules")

page_size = 1000
page_token = ""
initial_loop = True

# loop to makes sure we get all rules (even if there are > 1000 for some odd reason)
while page_token or initial_loop:
  url_params = {'page_size': page_size}
  if page_token:
    url_params["page_token"] = page_token
  encoded_url_params = urlencode(url_params)

  uri_to_post = f"https://{region_prefix}backstory.googleapis.com/v2/detect/rules?{encoded_url_params}"
  resp = http_client.request(uri_to_post, "GET")
  json_resp = json.loads(resp[1])
  if resp[0].status != http.HTTPStatus.OK:
    pprint(json_resp.get('error').get('message'))
  else:
    if 'rules' in json_resp.keys():
      for rule in json_resp["rules"]:
        with open(f"{tenant_code}_rules/{rule['ruleName']}.yaral", "w") as rule_file:
          print(f"Writing rule \"{rule['ruleName']}\" to {f'{tenant_code}_rules'} folder.")
          rule_file.write(rule['ruleText'])
    else: 
      print(f"There are no rules for customer code: {tenant_code}")
    initial_loop = False
    if "nextPageToken" in json_resp.keys():
      page_token = json_resp["nextPageToken"]
    else:
      page_token = False
    sleep(1) # pause for a second to make sure we don't hit API limits if there are multiple pages of rules.

if download_zipped_data:
  os.system(f"zip -r {tenant_code}_rules.zip {tenant_code}_rules")
  files.download(f"{tenant_code}_rules.zip")


In [ ]:
#@title Get all tenant's rules

#@markdown If selected a zip file with all the rules for all the tenants will be downloaded.
download_zipped_data = True #@param {type:"boolean"}

for tenant_code in mssp_customers:

  SERVICE_ACCOUNT_FILE = f"{tenant_code}.json"
  credentials = service_account.Credentials.from_service_account_file(SERVICE_ACCOUNT_FILE, scopes=SCOPES)
  http_client = _auth.authorized_http(credentials)

  if f"{tenant_code}_rules" not in os.listdir():
    os.mkdir(f"{tenant_code}_rules")

  page_size = 1000 
  page_token = ""
  initial_loop = True

  # loop to makes sure we get all rules (even if there are > 1000 for some odd reason)
  while page_token or initial_loop:
    url_params = {'page_size': page_size}
    if page_token:
      url_params["page_token"] = page_token
    encoded_url_params = urlencode(url_params)

    uri_to_post = f"https://{region_prefix}backstory.googleapis.com/v2/detect/rules?{encoded_url_params}"
    resp = http_client.request(uri_to_post, "GET")
    json_resp = json.loads(resp[1])
    if resp[0].status != http.HTTPStatus.OK:
      pprint(json_resp.get('error').get('message'))
    else:
      if 'rules' in json_resp.keys():
        for rule in json_resp["rules"]:
          with open(f"{tenant_code}_rules/{rule['ruleName']}.yaral", "w") as rule_file:
            print(f"Writing rule \"{rule['ruleName']}\" to {f'{tenant_code}_rules'} folder.")
            rule_file.write(rule['ruleText'])
      else: 
        print(f"There are no rules for customer code: {tenant_code}")
      initial_loop = False
      if "nextPageToken" in json_resp.keys():
        page_token = json_resp["nextPageToken"]
      else:
        page_token = False
      sleep(1) # pause for a second to make sure we don't hit API limits if there are multiple pages of rules.

if download_zipped_data:
  os.system(f"zip -r rules.zip *_rules")
  files.download(f"rules.zip")


In [ ]:
#@title VerifyRule
#@markdown Verifies that a rule is a valid YARA-L 2.0 rule without creating a new rule or evaluating it over data.

#@markdown Choose whether to upload a text file with the rule or use one already in the filesystem of the colab instance.

# reinitialize the mssp key because we don't know what key the current web client is using
SERVICE_ACCOUNT_FILE = "mssp_key.json"
credentials = service_account.Credentials.from_service_account_file(SERVICE_ACCOUNT_FILE, scopes=SCOPES)
http_client = _auth.authorized_http(credentials)

upload_rule = False #@param {type: "boolean"}
rule_filename = "blah.yaral" #@param {type: "string"}

if upload_rule:
  print("Please select and upload your keyfile")
  uploaded = files.upload()

  rule_filename = list(uploaded.keys())[0]

with open(rule_filename, "r") as rule:
  rule_text = rule.read()

request_body = {
    "ruleText" : rule_text
}

uri_to_post = f"https://{region_prefix}backstory.googleapis.com/v2/detect/rules:verifyRule"
resp = http_client.request(uri_to_post, "POST", body=json.dumps(request_body))
json_resp = json.loads(resp[1])
if resp[0].status != http.HTTPStatus.OK:
  pprint(json_resp.get('error').get('message'))
else:
  if "success" in json_resp.keys():
    print(f"Valid rule")
  else:
    print(f"Invalid rule... Context:\n{json_resp['context']}")
  


valid rule


In [ ]:
#@title Upload rules to a single tenant
tenant_code = "dmrsknfr" #@param {type:"string"}
#@markdown If the below option is checked then any newly created rules will be set to live (active). The default behavior is not to be live. 

#@markdown Any existing rules will remain in the state they were.
set_rules_to_live = False #@param {type:"boolean"}

SERVICE_ACCOUNT_FILE = f"{tenant_code}.json"
credentials = service_account.Credentials.from_service_account_file(SERVICE_ACCOUNT_FILE, scopes=SCOPES)
http_client = _auth.authorized_http(credentials)

# grab a list of the current rules, this will tell us if we want to update a rule or create a new one
page_size = 1000 
page_token = ""
initial_loop = True
tenant_rules = []
tenant_rules_by_name = {}
tenant_rules_by_id = {}
rule_counter = 0

while page_token or initial_loop:
  url_params = {'page_size': page_size}
  if page_token:
    url_params["page_token"] = page_token
  encoded_url_params = urlencode(url_params)

  uri_to_post = f"https://{region_prefix}backstory.googleapis.com/v2/detect/rules?{encoded_url_params}"
  resp = http_client.request(uri_to_post, "GET")
  json_resp = json.loads(resp[1])
  if resp[0].status != http.HTTPStatus.OK:
    pprint(json_resp.get('error').get('message'))
  else:
    if 'rules' in json_resp.keys():
      for rule in json_resp['rules']:
        tenant_rules.append(rule)
        # create lookups by both name and rule id because they'll be useful
        # tenant_rules_by_id[rule['ruleId']] = rule
        tenant_rules_by_name[rule['ruleName']] = rule
        rule_counter += 1
    else:
      print(f"No rules found for {tenant_code}")
    initial_loop = False
    if "nextPageToken" in json_resp.keys():
      page_token = json_resp["nextPageToken"]
    else:
      page_token = False
    sleep(1) # pause for a second to make sure we don't hit API limits if there are multiple pages of rules.

# time to upload the rules 
# going to ask for a zip file of rule(s)
upload_success = False
while not upload_success:
  print("Please select and upload your rule(s) zip file.")
  uploaded = files.upload()
  rule_filename = list(uploaded.keys())[0]
  if not bool(re.search("\.zip$", rule_filename, flags=re.IGNORECASE)):
    print(f"{rule_filename} does not appear to be a zip file (based on extension).")
  else:
    upload_success = True

print(f"Unzipping {rule_filename} to rules_to_push/ ...")
os.system(f"unzip -j {rule_filename} -d rules_to_push/")

possible_files_to_upload = os.listdir(f"rules_to_push")

for file in possible_files_to_upload:
  with open(f"rules_to_push/{file}", "r") as rule_file:
    rule_text = rule_file.read()

  rule_name_search_result = re.search("rule ([^\s]+) {", rule_text, re.IGNORECASE)
  if rule_name_search_result:
    rule_name = rule_name_search_result[1]
  else:
    print(f"ERROR: {file} does not appear to be a valid rule. Skipping it.\n")
    continue
  
  # validate possible rule
  request_body = {
      "ruleText" : rule_text
  }
  uri_to_post = f"https://{region_prefix}backstory.googleapis.com/v2/detect/rules:verifyRule"
  resp = http_client.request(uri_to_post, "POST", body=json.dumps(request_body))
  json_resp = json.loads(resp[1])
  if resp[0].status != http.HTTPStatus.OK:
    pprint(json_resp.get('error').get('message'))
    continue
  else:
    if "success" in json_resp.keys():
      print(f"SUCCESS: {rule_name} in {file} has been validated as proper Yara-L 2.0\n")
    else:
      print(f"ERROR: {rule_name} in {file} is an invalid rule... Context:\n{json_resp['context']}\n")
      continue
  
  # check to see if this rule name is already in the instance
  if rule_name in tenant_rules_by_name.keys():
    existing_rule_id = tenant_rules_by_name[rule_name]["ruleId"]

    uri_to_post = f"https://{region_prefix}backstory.googleapis.com/v2/detect/rules/{existing_rule_id}:createVersion"
    resp = http_client.request(uri_to_post, "POST", body=json.dumps(request_body))
    json_resp = json.loads(resp[1])
    if resp[0].status != http.HTTPStatus.OK:
      pprint(json_resp.get('error').get('message'))
    else:
      print(f"Rule name: {rule_name} \n\ton tenant: {tenant_code} \n\thas been updated.")
      continue
  else:
    # creating a new rule

    uri_to_post = f"https://{region_prefix}backstory.googleapis.com/v2/detect/rules"
    resp = http_client.request(uri_to_post, "POST", body=json.dumps(request_body))
    json_resp = json.loads(resp[1])
    if resp[0].status != http.HTTPStatus.OK:
      pprint(json_resp.get('error').get('message'))
    else:
      pprint(f"Created rule named: {rule_name} \n\ton tenant {tenant_code}")
      if set_rules_to_live:
        uri_to_post = f"https://{region_prefix}backstory.googleapis.com/v2/detect/rules/{json_resp['ruleId']}:enableLiveRule"
        resp = http_client.request(uri_to_post, "POST")
        json_resp = json.loads(resp[1])
        if resp[0].status != http.HTTPStatus.OK:
          pprint(json_resp.get('error').get('message'))
        else:
          print(f"Set rule named :{rule_name}\n\tto live status.")


# clean up time
os.system(f"rm {rule_filename}")
os.system(f"rm -rf rules_to_push")

        



In [ ]:
#@title Upload rules to all tenants

#@markdown If the below option is checked then any newly created rules will be set to live (active). The default behavior is not to be live. 

#@markdown Any existing rules will remain in the state they were.
set_rules_to_live = False #@param {type:"boolean"}

# time to upload the rules 
# going to ask for a zip file of rule(s)
upload_success = False
while not upload_success:
  print("Please select and upload your rule(s) zip file.")
  uploaded = files.upload()
  rule_filename = list(uploaded.keys())[0]
  if not bool(re.search("\.zip$", rule_filename, flags=re.IGNORECASE)):
    print(f"{rule_filename} does not appear to be a zip file (based on extension).")
  else:
    upload_success = True

print(f"Unzipping {rule_filename} to rules_to_push/ ...")
os.system(f"unzip -j {rule_filename} -d rules_to_push/")

possible_files_to_upload = os.listdir(f"rules_to_push")

for tenant_code in mssp_customers:

  SERVICE_ACCOUNT_FILE = f"{tenant_code}.json"
  credentials = service_account.Credentials.from_service_account_file(SERVICE_ACCOUNT_FILE, scopes=SCOPES)
  http_client = _auth.authorized_http(credentials)

  # grab a list of the current rules, this will tell us if we want to update a rule or create a new one
  page_size = 1000 
  page_token = ""
  initial_loop = True
  tenant_rules = []
  tenant_rules_by_name = {}
  tenant_rules_by_id = {}
  rule_counter = 0

  while page_token or initial_loop:
    url_params = {'page_size': page_size}
    if page_token:
      url_params["page_token"] = page_token
    encoded_url_params = urlencode(url_params)

    uri_to_post = f"https://{region_prefix}backstory.googleapis.com/v2/detect/rules?{encoded_url_params}"
    resp = http_client.request(uri_to_post, "GET")
    json_resp = json.loads(resp[1])
    if resp[0].status != http.HTTPStatus.OK:
      pprint(json_resp.get('error').get('message'))
    else:
      if 'rules' in json_resp.keys():
        for rule in json_resp['rules']:
          tenant_rules.append(rule)
          # create lookups by both name and rule id because they'll be useful
          # tenant_rules_by_id[rule['ruleId']] = rule
          tenant_rules_by_name[rule['ruleName']] = rule
          rule_counter += 1
      else:
        print(f"No rules found for {tenant_code}")
      initial_loop = False
      if "nextPageToken" in json_resp.keys():
        page_token = json_resp["nextPageToken"]
      else:
        page_token = False
      sleep(1) # pause for a second to make sure we don't hit API limits if there are multiple pages of rules.

  for file in possible_files_to_upload:
    with open(f"rules_to_push/{file}", "r") as rule_file:
      rule_text = rule_file.read()

    rule_name_search_result = re.search("rule ([^\s]+) {", rule_text, re.IGNORECASE)
    if rule_name_search_result:
      rule_name = rule_name_search_result[1]
    else:
      print(f"ERROR: {file} does not appear to be a valid rule. Skipping it.\n")
      continue
    
    # validate possible rule
    request_body = {
        "ruleText" : rule_text
    }
    uri_to_post = f"https://{region_prefix}backstory.googleapis.com/v2/detect/rules:verifyRule"
    resp = http_client.request(uri_to_post, "POST", body=json.dumps(request_body))
    json_resp = json.loads(resp[1])
    if resp[0].status != http.HTTPStatus.OK:
      pprint(json_resp.get('error').get('message'))
      continue
    else:
      if "success" in json_resp.keys():
        print(f"SUCCESS: {rule_name} in {file} has been validated as proper Yara-L 2.0\n")
      else:
        print(f"ERROR: {rule_name} in {file} is an invalid rule... Context:\n{json_resp['context']}\n")
        continue
    
    # check to see if this rule name is already in the instance
    if rule_name in tenant_rules_by_name.keys():
      existing_rule_id = tenant_rules_by_name[rule_name]["ruleId"]

      uri_to_post = f"https://{region_prefix}backstory.googleapis.com/v2/detect/rules/{existing_rule_id}:createVersion"
      resp = http_client.request(uri_to_post, "POST", body=json.dumps(request_body))
      json_resp = json.loads(resp[1])
      if resp[0].status != http.HTTPStatus.OK:
        pprint(json_resp.get('error').get('message'))
      else:
        print(f"Rule name: {rule_name} \n\ton tenant: {tenant_code} \n\thas been updated.")
        continue
    else:
      # creating a new rule

      uri_to_post = f"https://{region_prefix}backstory.googleapis.com/v2/detect/rules"
      resp = http_client.request(uri_to_post, "POST", body=json.dumps(request_body))
      json_resp = json.loads(resp[1])
      if resp[0].status != http.HTTPStatus.OK:
        pprint(json_resp.get('error').get('message'))
      else:
        pprint(f"Created rule named: {rule_name} \n\ton tenant {tenant_code}")
        if set_rules_to_live:
          uri_to_post = f"https://{region_prefix}backstory.googleapis.com/v2/detect/rules/{json_resp['ruleId']}:enableLiveRule"
          resp = http_client.request(uri_to_post, "POST")
          json_resp = json.loads(resp[1])
          if resp[0].status != http.HTTPStatus.OK:
            pprint(json_resp.get('error').get('message'))
          else:
            print(f"Set rule named :{rule_name}\n\tto live status.")

  # clean up time
  os.system(f"rm {rule_filename}")
  os.system(f"rm -rf rules_to_push")

          

